# Generative AI - Combient Mix AB / Silo AI for Patricia AB 2023

Prompt Engineering (PE) is the primary vehicle for guiding generative AI models towards stable applications. It is particularly applicable to Large Language Models (LLMs) and involves formulating prompts and prompting schemas in order to retrieve appropriate responses to queries. It is essential for meaningful interactions with LLMs.

This notebook provides a hands-on introduction to PE for end users with a basic understanding of the Python programming language. No advanced coding and no technical background knowledge of generative AI or LLM is required. The notebook is divided into three sections, each corresponding to material covered in the first of two live seminars. There will be ~40 minutes available in order to go through and complete each section.
<br />
<br />

***

**Structure**

Exercise **1 & 2 during the first session** (~ 40 minutes per exercise) and exercise **3 during the second session** (~ 60 minutes)


1.) First, we'll explore the basics of prompting LLMs and the importance of PE. We will see examples of some important and cutting edge techniques for prompting efficiently and with intent
* 0 / 1 / few-shot prompting,
* Role-Task-Format (RTF),
* In-Context-Learning (ICL),
* Chain-of-Thought (CoT) and
* Tree-of-Thought (ToT)

2.) We'll learn how to integrate and utilize existing tools for specific tasks. We focus on some of the tools available via enabling plugins for GPT-4 at [OpenAI chat interface](https://chat.openai.com/). Example tasks include 
  * **Searching the internet for recent information** LLMs are generally pre-trained and thus have a knowledge cutoff at the time when training stopped. The latest GPT-4 model of OpenAI for example has a knowledge cutoff at January 2022. This means that it was not trained on any data produced after that time, so if we want to include updated information this must be done by fetching the information and providing it as context together with our query.
  * **Using your own PDF documents as a knowledge base** (plugin: AskYourPDF). There are several plugins available for GPT-4 which provide the capability of uploading a PDF document and use it as a reference for interacting with the LLM.

3.) The third exercise is more programming heavy and involves working with API calls. We'll guide you through it! This allows us to perform more complex tasks and write our own custom functions utilizing the power of the LLM. We introduce the Python library [Langchain](https://python.langchain.com/docs/get_started/introduction.html) and some of its tools for manipulating prompts. This allows for more intricate problem-solving as well as gaining control over the reliability of the output. We will see how to use this in order to accomplish Retrieval Augmented Generation (RAG) from our own knowledge base (the PDF document). This exercise demonstrates how the plugins used in exercise 2 work under the hood. Examples of what we can achieve include

* constraining the system behaviour, for example to mitigate hallucinations from the LLM
* Retrieval Augmented Generation from an external knowledge base such as the internet or a collection of documents

This provides a deeper understanding of the fundamental components essential for constructing advanced generative AI systems.

***

**Note:**

In order to run this notebook properly you will need

* **Gmail account** - Follow the provided instructions to download the notebook to your GDrive, so that you can edit and save it freely.
* **Google GDrive access** - enable the app `Google Colaboratory`. The process is highlighted in the accompanyiing reading material, but you can also ask ChatGPT a question like this by posing the following query <span style="color:blue;">how do I enable google colab for the first time?</span>

> Run the below code blocks to install necessary packages.
>
> Notebook code blocks can be executed via either: 
> * **shift + enter**: executes current code block and moves to the cell below
> * **control + enter**: executes current code block

## Environment setup

Here we set up the environment and make sure we can access data via Google Drive.

Run the code below to load the GenAI map we will be using during this course. The code is fetching a map called GenAI from our Github repository. Click on plus next to the cell to run the code.

In [1]:
use_drive = False

if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive

  !git clone https://github.com/statisticalmodel/GenAI

You can now view the files by clicking on the map and follow the path down in the GenAI short course.

![Image of GitClone](https://github.com/statisticalmodel/GenAI/blob/main/GenAI%20short%20course/GitClone.png?raw=true)

## Packages & Imports

Installing and importing necessary packages/modules. Note that these should be installed only in a virtual environment when using the Colab Notebook.

Run the below code block to install some of the Python libraries which are required for running the Notebook.

Note that this may take up to ~15 seconds to complete.

> **NB: All packages below are not necessary and should be cleaned after completion of exercises**

In [2]:
!pip install -qU transformers \
    --upgrade huggingface_hub \
    -q sentencepiece \
    -q accelerate \
    -q tiktoken \
    -q openai \
    -q langchain \
    -q sentence-transformers \
    -q jq \
    -q faiss-cpu \
    -q pypdf \
    -q wikipedia \
    -q duckduckgo-search \
    -q colorama \
    -q PyMuPDF 

Run the below code block to import the necessary library modules used in the notebook.

Note that this may take up to ~20 seconds to complete.

> **NB: All packages below are not necessaty and should be cleaned after completion of exercises**

In [3]:
# Some system and base modules
import os
import sys
from pathlib import Path
from timeit import default_timer as timer
from typing import Any, List, Dict, Optional, Type
import getpass
import numpy as np
import pandas as pd
import json
import requests
from PIL import Image
import io
import re
from time import time
from termcolor import colored

# NLP modules
import torch
import openai
from openai.embeddings_utils import cosine_similarity
from huggingface_hub import login
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,JSONLoader, UnstructuredMarkdownLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
import fitz

# Other modules
from colorama import Fore, Back, Style
import wikipedia

# Modules for plotting
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

/Users/frida/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import helper functions used in the Notebook

Run the code block below to import customized helper functions used in the notebook. These functions have been written to, for example, process PDF files and reduce clutter in the notebook by using explicit code. All of the imported functions are located in the file `helper_functions_E3.py` where you can review their definitions if you're interested.

In [4]:
from helper_functions import *

## Setting the access key for OpenaAI API

**The OpenAI API key can be set manually in the notebook by runnning the code cell block below.**

You can optionally set it as an environment variable; by typing the following in your Mac terminal
```
export OPENAI_API_KEY=sk-...
```
or if you are using windows 10 you can type the following in a Command window
```
set OPENAI_API_KEY=sk-...
```
Observe the lack of space in the value designations.

> **NB: Delete the API access key below before pushing to repo etc.**

In [5]:
# sk-...

# Here we can set the OpenAI api access key manually in case it fails to load from the environment.
if not os.environ.get("OPENAI_API_KEY"):
    api_key = getpass.getpass("Enter OpenAI API Key here")
    os.environ["OPENAI_API_KEY"] = api_key
else:
  print(f"OPENAI_API_KEY fetched from environment!")


#print(f"The Open AI access key is given by: \n\n {os.environ['OPENAI_API_KEY']}")

# Hands-On 3: Customized Tools for Extended LLM Functionality

Using LangChain agents for tasks. Example themes for exercises:

* Search the internet, e.g., Wikipedia, DuckDuckGo or Google Custom Search APIs.
  - Useful and easy to implement and understand at a basic level.
  - Allows to go beyond the Wiki app and have more control.
* Search a knowledge base, e.g., PDF, Excel or plain text files.
  - Useful for many purposes. Builds on previous exercises and showcase the behind the scenes work involved in the plugins.
  - Requires that we pre-vet or supply the material since there may be format issues which can't be dealt with in a timely fashion otherwise.
* Generate and understand code - maybe some simple examples, but only if there is time and we can think of some relatively simple and instructive examples.

## Intro to chatting via the API (Application Program Interface)

In order to instantiate the chat interface with OpenAI models, we can call it through the API below. Model is specified via `model` on the second row, and should be a valid name within single or double quotation marks so that it is interpreted as a string object. Available [models to choose from are](https://platform.openai.com/docs/models/overview)

* **gpt-3.5-turbo** uses the latest available model version of GPT-3.5. Allows for up to **4,097 tokens** as input and output
    * **gpt-3.5-turbo-0613** is the latest version of GPT-3.5, currently the same as above
    * **gpt-3.5-turbo-16k** is a version of GPT-3.5 with longer context window, up to **16,385 tokens**

</b>


* **gpt-4** uses the latest available model version of GPT-4. Allows using **8,192 tokens** as input and output
    * **gpt-4-0613** is the latest version of GPT-4, same as above

We implement the chat interface here via a wrapper function which we import from the `langchain` library. This does not return all of the output available from the models, such as token counts, but allow for a simpler and more streamlined interface for chatting. The `temperature` parameter determines the randomness or stochastic nature of the output with a value of 0 being fully deterministic with the same output for repeated queries. Experience from human evaluations have shown that most people prefer the output to be somewhat creative in nature with a temperature value of around 0.7, which is the default value set for the OpenAI models. You can try changing this value and experience how the output changes for the same query.

> **NB 1: The response time of the GPT-3.5 model is typically between 15-45 seconds. If the cell runs for a minute, then stop the execution and retry instead of waiting.**

> **NB 2: This is one option for implementing the LLM call, which is nice due to its simple and intuitive format. At the bottom of this section we explore using the OpenAI API directly as an alternative option, which provides greater control of the input/output.**

In [6]:
from langchain.chat_models import ChatOpenAI

# This createss an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo-0613",
    temperature=0.7
)

We also specify messages according to a schema used by various functions in the `langchain` library. This will facilitate some of the more advanced functionalities we will make use of later. Messages in a chat conversation are generally provided as a `list` (inside square brackets) of conversation turns.

In [111]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = "You are a helpful assistant."

# The user prompt is what would be written in the chat interface, your query
user_prompt = "What do you know about Patricia Industries from Sweden?"

# These are collected into a message list of
#
# *  SystemMessage - the system prompt
# *  HumanMessage - the user query
# *  AIMessage - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [8]:
# We can check what the messages look like by printing it out
messages

[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='What do you know about Patricia Industries from Sweden?')]

Once we have a list of messages in the above format it is easy to call the OpenAI model and get a response

In [9]:
# Here we collect the ouput from the chat model in a variable response
response = chat_model(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

Patricia Industries is an investment company based in Sweden. It is a part of the Swedish industrial group, Investor AB. 

Patricia Industries focuses on long-term investments in companies with strong market positions and growth potential. Its investment strategy is centered around three main areas: Nordics, DACH (Germany, Austria, and Switzerland), and North America. 

The company seeks to actively develop and support its portfolio companies by providing strategic guidance and expertise. It aims to create value through operational improvements, strategic acquisitions, and investments in innovation and sustainability.

Patricia Industries has a diverse portfolio of companies across various industries, including healthcare, industrials, consumer goods, and technology. Some notable investments include Mölnlycke Health Care, Aleris, Permobil, BraunAbility, and Vectura Group.

Overall, Patricia Industries is known for its long-term approach to investing, commitment to sustainable growth, a

The full response is actually formatted as a `AIMessage` as we can see by printing out the full response without using the print function. Observe that if don not use the formatting of the `print` fucntion you will see linebreak characters such as `\n` appearing in the message.

In [10]:
response

AIMessage(content='Patricia Industries is an investment company based in Sweden. It is a part of the Swedish industrial group, Investor AB. \n\nPatricia Industries focuses on long-term investments in companies with strong market positions and growth potential. Its investment strategy is centered around three main areas: Nordics, DACH (Germany, Austria, and Switzerland), and North America. \n\nThe company seeks to actively develop and support its portfolio companies by providing strategic guidance and expertise. It aims to create value through operational improvements, strategic acquisitions, and investments in innovation and sustainability.\n\nPatricia Industries has a diverse portfolio of companies across various industries, including healthcare, industrials, consumer goods, and technology. Some notable investments include Mölnlycke Health Care, Aleris, Permobil, BraunAbility, and Vectura Group.\n\nOverall, Patricia Industries is known for its long-term approach to investing, commitme

We can incorporate the `AIMessage` response in a new series of messages and also ask a follow up question if you wish. This provides the illusion of the chatmodel having a memory and being able to continue a conversation for a few turns, as you have seen while working in the browser environment.

In [112]:
# Let's ask a follow-up question which refers to the previous conversation without explicitly mentioning e.g., Patricia Industries
follow_up_question = "Can you say something more about the investment approach?"

# We can now add the respinse we got from the first query together with our follow-up question
messages = messages + [response, HumanMessage(content=follow_up_question)]
messages

[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='What do you know about Patricia Industries from Sweden?'),
 AIMessage(content='The strategies for adjusting the Front Seat Height on Slipstream Single-Sided Forks are similar in both sets of instructions. Both sets mention the use of a 5/8" Open End Wrench and a screwdriver as the required tools for the adjustment. \n\nAdditionally, both sets of instructions mention the steps of removing the caster, removing the E-Ring, removing the axle, placing the axle in the alternate hole, replacing the E-Ring, and finally replacing the caster. \n\nHowever, there are some differences in the instructions. The first set of instructions mentions the need for protective eyewear when removing the E-Ring, while the second set does not. The second set of instructions also includes additional tools such as a Wood Block, Drafting Triangle, Rubber Mallet, and Ruler or Measuring Tape, which are not mentioned in the first set of in

In [113]:
# Here we collect the ouput from the chat model in a variable response
response = chat_model(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

Patricia Industries is an investment firm based in Sweden that follows a long-term and active ownership approach. They focus on investing in companies with strong growth potential and a sustainable business model. 

Their investment approach involves partnering with companies and providing them with the necessary resources, expertise, and support to drive growth and value creation. They aim to be a strategic and value-adding owner, working closely with management teams to develop and execute on growth strategies.

Patricia Industries has a sector-agnostic approach, meaning they invest in a wide range of industries including healthcare, technology, industrials, and consumer goods. They seek to identify companies that have a competitive advantage, innovative products or services, and a strong market position.

In addition to financial capital, Patricia Industries also brings operational expertise and a network of industry contacts to help their portfolio companies succeed. They have a lo

By continuing in this fashion we can record a conversation history which we send to the chat model as long as the total text content does not exceed the models content limit.

### OPTIONAL: Using the OpenAI API directly instead of the `langchain` wrapper

The below code snippet calls the OpenAI API directly, which allow to access all of the available input and output options. This provides more control but is less intuitive and does not integrate as directly with other functions from langchain.

In [13]:
# Needs to be set for the OpenAI API to be callable
openai.api_key = os.getenv("OPENAI_API_KEY")


# Model and parameters (almost all that are available, logit_bias not included). Parameters model and messages are required
model = "gpt-3.5-turbo-0613"
max_tokens = 1024         # Max nr of tokens to generate in the chat completion, limited by model choice
temperature = 0.7         # Value in (0, 2). Sampling temperature for stochastic nature in response
top_p = 1                 # Vale in (0, 1). Nucleus sampling, optional to temperature, considers tokens comprising top_p probability mass
frequency_penalty = 0     # Value in (-2, 2). Positive value penalizes new tokens based on frequency in text so far, to decrease likelihood of repeating sentences
presence_penalty = 0      # Value in (-2, 2). Positive value penalizes new tokens if appeared in text so far, to increase likelihood of switching to new topics
n=1                       # How many completion options to generate for each message
stream = False            # Boolean value which can allow for streaming response



function_call = "none"      # 'auto' if call function or generate message, supply {"name": my_func}
functions = [{"name": "Name", "description": "semantic description of what functions do", "parameters": {"type": "object", "properties": {}}}]

# Messages are similar to the langchain format but different
# Other values for the key 'role' are 'assistant' and 'function'
system_prompt = "You are a helpful assistant."
user_prompt = "What do you know about Patricia Industries from Sweden?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
    ]


# API call
response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
            n=n,
            stream=stream,
            function_call=function_call,
            functions=functions
        )


# Printing the response requires a slighty less intuitive format
print(response.choices[0].message["content"])

Patricia Industries is a Swedish investment company that operates as a part of Investor AB, one of Sweden's largest industrial holding companies. Patricia Industries focuses on long-term investments in companies across various sectors, including healthcare, technology, and consumer goods. The company aims to support and develop its portfolio companies by providing strategic guidance and financial resources. Patricia Industries has a strong commitment to sustainability and responsible business practices.


We can check what the full response contains without any parsing of the output. By printing the below you will see that the response contain information about the token usage etc. which are crucial for estimating the cost of running queries

In [14]:
response

<OpenAIObject chat.completion id=chatcmpl-8FgDJVOpRPp7WbmW3jpYM3JS8TfHx at 0x2b186c810> JSON: {
  "id": "chatcmpl-8FgDJVOpRPp7WbmW3jpYM3JS8TfHx",
  "object": "chat.completion",
  "created": 1698747881,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Patricia Industries is a Swedish investment company that operates as a part of Investor AB, one of Sweden's largest industrial holding companies. Patricia Industries focuses on long-term investments in companies across various sectors, including healthcare, technology, and consumer goods. The company aims to support and develop its portfolio companies by providing strategic guidance and financial resources. Patricia Industries has a strong commitment to sustainability and responsible business practices."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 55,
    "completion_tokens": 79,
    "total_tokens": 134
  }
}

## Get search results from internet

We will explore two simple open source options for adding internet content as context for our query to the LLM:

* The Wikipedia API which allow us to read in a relevant Wikipedia information to get factual information on a topic
* The DuckDuckGo API which allows us to search updated information on a topic, to use as context for our query

### Wikipedia API

If we want to search for or summarize facts from Wikipedia and know the page title it is easy to use the Wikipedia API to get the content from that page. Be aware that the Wikipedia API is not a general purpose search function and relies on the query matching an existing page title relatively well.

In [15]:
# Let's try to ask a question
query = "What is Patricia Industries"

# Try searching with a general question. The how_many argument specifies how many letters should be printed from the response
result_wiki = wiki_search_api(query, how_many=1000)
print(result_wiki)

https://en.wikipedia.org/wiki/Patricia_Arquette
Patricia Arquette (born April 8, 1968) is an American actress. She made her feature film debut as Kristen Parker in A Nightmare on Elm Street 3: Dream Warriors (1987) and has since starred in several film and television productions. She has received several awards including an Academy Award, two Primetime Emmy Awards and three Golden Globe Awards.
She had starring roles in several critically acclaimed films, including True Romance (1993), Ed Wood (1994), Flirting with Disaster (1996), Lost Highway (1997), The Hi-Lo Country (1998), and Bringing Out the Dead (1999). From 2005 to 2011, she starred as a character based on the medium Allison DuBois in the supernatural drama series Medium, winning the Primetime Emmy Award for Outstanding Lead Actress in a Drama Series in 2005.
For playing a single mother in the coming-of-age film Boyhood (2014), which was filmed from 2002 until 2014, Arquette won the Academy Award for Best Supporting Actress. F

As we see the answer is not related to Patricia Industries at all. By searching instead for a keyword we can increase the chances of obtaining a more relevant answer.

In [16]:
query_keyword = "Patricia Industries"

# By searching on the keyword we obtain a better match. The how_many argument specifies how many letters should be printed from the response
result_wiki = wiki_search_api(query_keyword, how_many=1000)
print(result_wiki)

https://en.wikipedia.org/wiki/Investor_AB
Investor AB is a Swedish investment and holding company, often considered a de facto conglomerate. One of Sweden's largest companies, Investor AB serves as the investment arm of the prominent Swedish Wallenberg family; the family's companies are involved in a variety of industries, of which the primary industries are pharmaceuticals, telecommunications and industry.Investor AB is Sweden's most valuable publicly traded company; it has major or controlling holdings in several of Sweden's other largest companies. It has major investments worldwide through Patricia Industries and EQT Partners.


== History ==
Investor AB was established in Stockholm when new Swedish legislation made it more difficult for banks to own stocks in industrial companies on a long-term basis. The shareholdings of the Wallenberg family bank, Stockholms Enskilda Bank, were transferred to Investor AB, a newly formed industrial holding company spun off from the bank, which wo

We can customize this somewhat by adding a function which extracts the keyword from a sentence by looking for the phrase `keyword: <KEYWORD>`. This has some limitiations though since it requires us to include this kind of formatted phrase in our query.

In [17]:
query_keyword = "Patricia Industries"
query_with_keyword = f"What is keyword: {query_keyword}"

# By searching on the keyword we obtain a better match. The how_many argument specifies how many letters should be printed from the response
result_wiki = wiki_search_api(find_keyword(query_with_keyword), how_many=1000)
print(result_wiki)

https://en.wikipedia.org/wiki/Investor_AB
Investor AB is a Swedish investment and holding company, often considered a de facto conglomerate. One of Sweden's largest companies, Investor AB serves as the investment arm of the prominent Swedish Wallenberg family; the family's companies are involved in a variety of industries, of which the primary industries are pharmaceuticals, telecommunications and industry.Investor AB is Sweden's most valuable publicly traded company; it has major or controlling holdings in several of Sweden's other largest companies. It has major investments worldwide through Patricia Industries and EQT Partners.


== History ==
Investor AB was established in Stockholm when new Swedish legislation made it more difficult for banks to own stocks in industrial companies on a long-term basis. The shareholdings of the Wallenberg family bank, Stockholms Enskilda Bank, were transferred to Investor AB, a newly formed industrial holding company spun off from the bank, which wo

**Try it out yourself:**

Try it out by searching with your own keyword. The more closely this matches an existing Wikipedia title page, the more likely you are of finding what you look for.

In [18]:
# Specify a query keyword argument. The more closely this match an existing Wikipedia title page, the more likely you are of finding what you look for
query_keyword = "..."

# Search on the keyword. The how_many argument specifies how many letters should be printed from the response
result_wiki = wiki_search_api(query_keyword, how_many=1000)
print(result_wiki)

https://en.wikipedia.org/wiki/Ellipsis
The ellipsis ... (; also known informally as dot dot dot) is a series of dots that indicates an intentional omission of a word, sentence, or whole section from a text without altering its original meaning. The plural is ellipses. The term originates from the Ancient Greek: ἔλλειψις, élleipsis meaning 'leave out'.Opinions differ as to how to render ellipses in printed material. According to The Chicago Manual of Style, it should consist of three periods, each separated from its neighbor by a non-breaking space: . . .. According to the AP Stylebook, the periods should be rendered with no space between them: .... A third option is to use the Unicode character U+2026 … HORIZONTAL ELLIPSIS.


== Background ==
The ellipsis is also called a suspension point, points of ellipsis, periods of ellipsis, or (colloquially) "dot-dot-dot". Depending on their context and placement in a sentence, ellipses can indicate an unfinished thought, a leading statement, a s

#### Explicitly using Wikipedia facts for the purpose of answering a question

In [19]:
# This createss an instance of the model interface which we can subsequently call on
# Here you can try playing with the temperature parameter
temperature = 0.0

chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo",
    temperature=temperature
)

# We define a function for more easily call the LLM using the same prompt template but with a different query and context
def LLM_with_wiki_prompt(query: str, query_keyword: str, how_many: int=2000, temperature: float=0.0):

    # Retrieve serch results based the query
    context = wiki_search_api(query_keyword, how_many=how_many)

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt inputs the query and context
    user_prompt = f"""
    Consider carefully the following wiki facts as context.
    context: {context}

    Use only the above context and nothing else to answer the following query and summarize your response.
    query: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those bullets verbatim and do not try to rephrase them.
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model(messages, temperature=temperature)
    print(response.content)

In [20]:
query = "What is the relationship between Patricia Industries and Investor AB?"
query_keyword = "Patricia Industries"

LLM_with_wiki_prompt(query=query, query_keyword=query_keyword, temperature=temperature)

https://en.wikipedia.org/wiki/Investor_AB
- Investor AB has major investments worldwide through Patricia Industries.
- Patricia Industries is a subsidiary or division of Investor AB.
- Patricia Industries is the investment arm of Investor AB for its global investments.
- Patricia Industries is involved in various industries, including pharmaceuticals, telecommunications, and industry, just like Investor AB.
- Patricia Industries is responsible for managing and overseeing Investor AB's major investments outside of Sweden.


**Try it out yourself:**

Try it out by searching with your own query and relevant keyword. The more closely the keyword matches an existing Wikipedia title page, the more likely it is that the LLM will be able to answer your query based on Wikipedia facts.

In [21]:
query = "..."
query_keyword = "..."

LLM_with_wiki_prompt(query=query, query_keyword=query_keyword, temperature=temperature)

https://en.wikipedia.org/wiki/Ellipsis
The ellipsis, also known as dot dot dot, is a series of dots that indicates an intentional omission of a word, sentence, or whole section from a text without altering its original meaning. It can be rendered as three periods with a non-breaking space between them ( . . . ) according to The Chicago Manual of Style, or with no space between them (....) according to the AP Stylebook. Another option is to use the Unicode character U+2026 (… HORIZONTAL ELLIPSIS).

Ellipses can indicate an unfinished thought, a leading statement, a slight pause, an echoing voice, or a nervous or awkward silence. They can be used to trail off into silence or suggest melancholy or longing when placed at the end of a sentence.

The most common forms of an ellipsis are a row of three periods or the horizontal ellipsis glyph. Style guides have their own rules for the use of ellipses, such as using spaces around the periods or putting them together with a space before and aft

### DuckDuckGo search API

The [DuckDuckGo browser](https://duckduckgo.com/) provides an open source API which can be used to browse and search the internet. This returns a search response to a query and can be useful for incorporating a small amount of updated information as context to your LLM queries. More advanced search functionalities are possible but quickly become more technically involved for the user, so for the purpose of this short course we restrict to the basic search functionality.

In [22]:
query = "What's the latest business news about Investor AB?"

# Note that the retrieved responses are not necessarily the same every time we run a search.
response = get_search_results_ddg(query, max_results=50)
print(response)

Investor AB (publ) is a venture capital firm specializing in mature, middle market, buyouts and growth capital investments. It is operating through four business areas including core, private equity, operating, and financial investments. 920 followers $18.97 0.00 ( 0.00%) 3:06 PM 10/06/23 Pink Limited Info | $USD | Delayed Summary Ratings Financials Earnings Dividends Valuation Growth Profitability Momentum Peers Options Charting... Investor AB is a Sweden-based industrial holding company. Its operations are divided into three business segments: Listed Core Investments, EQT and Patricia Industries. The Listed Core Investments segment consists of listed holdings, which embrace ABB, AstraZeneca, Atlas Copco, Electrolux, Ericsson, Husqvarna, Nasdaq, Saab, SEB, Sobi and Wartsila. Conclusion: Summary: I'm long Investor AB because of these reasons: 100 years of successful investing. Investor AB has outperformed the Swedish index. The Wallenberg family controls the company and has proven them

Note that you receive a collection of relevant snippets to your query and not an actual answer. To collect the retrieved information into an answer we can use the LLLM.

#### Explicitly using the query for the purpose of answering a question

In [60]:
query = "What's the latest business news about Investor AB and its board?"

# Let's prepare to use the response as context for querying the LLM
context_ddg = get_search_results_ddg(query, max_results=50)

In [61]:
print(context_ddg)

Investor AB INVEb.ST LSEG Official Data Partner Latest Trade trading lower199.56SEK Change -0.42 % Change -0.21%Negative As of Oct 23, 2023. STOCKHOLM--Investor AB Chief Executive Johan Forssell will step down from his position in conjunction with the annual general meeting on May 7, the company said Friday. Through a consultancy agreement, Forssell will transition to a new role that will see him assigned to Investor as an industrial advisor with a focus on industrial companies ... Investor AB : News, information and stories for Investor AB | Nasdaq Stockholm: INVE B | Nasdaq Stockholm ... Business Leaders. Sectors. All our articles. Most Read News. ... CEO Johan Forssell will leave Investor in May 2024 in a new role with focus on Oct. 20. AQ Johan Forssell to Leave from Investor AB as President and Director, Effective 7 ... 920 followers $18.97 0.00 ( 0.00%) 3:06 PM 10/06/23 Pink Limited Info | $USD | Delayed Summary Ratings Financials Earnings Dividends Valuation Growth Profitability

Now we are ready to implement the internet search functionality together with the LLM for answering our queries.

In [62]:
# This createss an instance of the model interface which we can subsequently call on
# Here you can try playing with the temperature parameter
temperature=0.0


chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo",
    temperature=temperature
)

# We define a function for more easily call the LLM using the same prompt template but with a different query and context
def LLM_with_search_prompt(query: str, max_results: int=10, temperature: float=0.0):

    # Retrieve serch results based the query
    context = get_search_results_ddg(query, max_results=max_results)

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt inputs the query and context
    user_prompt = f"""
    One of our portfolio companies have handed us these news reports as context.
    context: {context}

    Use only the above context and nothing else to answer the following query and summarize your response.
    query: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those bullets verbatim and do not try to rephrase them.
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model(messages, temperature=temperature)
    print(response.content)

Running the above function will print out the response

In [63]:
LLM_with_search_prompt(query=query, temperature=temperature)

- Investor AB CEO Johan Forssell will step down from his position in conjunction with the annual general meeting on May 7.
- Johan Forssell will leave Investor AB in May 2024 in a new role with a focus on October 20.
- Investor AB is a venture capital firm specializing in mature, middle market, buyouts, and growth capital investments.
- Investor AB operates through four business areas including core, private equity, operating, and financial investments.


**Try it out yourself:**

Try it out by searching with your own query. The context will be automatically included via the prompt function defined above.

In [27]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."

LLM_with_search_prompt(query=query, temperature=temperature)

Based on the given context, it seems that the query is missing. There is no specific question mentioned in the context. Therefore, I cannot provide a response or summarize the answer.


## PDF knowledge base

We start by providing the location and names of the PDF files which we will be exploring.

In [28]:
# Absolute or relative path to folder where the PDF files are located
DATAPATH = "../Data/"

# Name of the PDF files (without the trailing .pdf file endings)
Patricia_permobil_manual = "Patricia_permobil_manual"
caterpillar_10k = "Caterpillar-10k"
whirlpool_10k = "whirlpool-10k"
electrolux_ann_rep = "Electrolux-annual-report"

# We collect all the PDF file names in a list
pdf_files = [
    Patricia_permobil_manual,
    caterpillar_10k,
    whirlpool_10k,
    electrolux_ann_rep
]

> **We load and inspect the text converted PDF files and some properties below**

We can now load one of the PDFs and examine its content. Note that this relies on extracting the content of the PDF as text and works well for the parts which are written text in the PDF, but works less well for e.g., tables and figures. In order to handle tables and figures well we typically need to perform pre-processing steps which are more complex and tailored for the specific PDF file type we wish to examine.

In [114]:
# Here we read in the PDF as text using a file from the helper function text_from_pdf()
pdf_text = text_from_pdf(DATAPATH + Patricia_permobil_manual + ".pdf")

# Here we count and print out the total nr of pages in the retrieved PDF
print(f"{Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}Nr of pages in pdf document:{Style.RESET_ALL} {len(pdf_text.keys())} \n")

# Here we count and print out the total nr of symbols in the retrieved PDF file
concat_text = pdf_dict_to_str(text_from_pdf(DATAPATH + Patricia_permobil_manual + ".pdf"))
print(f"The document contains {Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}{len(concat_text)}{Style.RESET_ALL} symbols")

# We then print an example page from the PDF
print(f"{Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}Ex text from first page:{Style.RESET_ALL}\n")
print(f"{pdf_text['page_1']}")

Nr of pages in pdf document: 96 

The document contains 195303 symbols
Ex text from first page:

IWARNING - READ THIS MANUAL 
DO NOT OPERATE THIS WHEELCHAIR WITHOUT FIRST READING AND UNDERSTANDING 
THIS OWNER’S MANUAL. IF YOU ARE UNABLE TO UNDERSTAND THE WARNINGS, 
CAUTIONS AND INSTRUCTIONS, CONTACT YOUR TiLITE DEALER OR TiLITE CUSTOMER 
SUPPORT AT (800) 545-2266 BEFORE ATTEMPTING TO USE THIS WHEELCHAIR.  IF  
YOU IGNORE THIS WARNING, YOU MAY FALL, TIP OVER OR LOSE CONTROL OF THE 
WHEELCHAIR AND SERIOUSLY INJURE YOURSELF OR OTHERS OR DAMAGE THE 
WHEELCHAIR. 
IWARNING - WHEELCHAIR SELECTION 
TiLITE MANUFACTURES A WIDE VARIETY OF WHEELCHAIRS TO MEET THE VARIED 
NEEDS OF WHEELCHAIR USERS. HOWEVER, TiLITE IS NOT YOUR HEALTH CARE 
ADVISOR, AND WE KNOW NOTHING ABOUT YOUR INDIVIDUAL CONDITION OR NEEDS. 
THEREFORE, THE FINAL SELECTION OF THE PARTICULAR MODEL, AND HOW IT IS 
ADJUSTED, AND THE TYPE OF OPTIONS AND ACCESSORIES NECESSARY REST SOLELY 
WITH YOU, THE WHEELCHAIR USER, AND THE HEALTH CA

### Following section performs chunking of text and creates indexed embeddings for the chunks

> **If possible we should probably either remove this section or make it optional**

Here we split up PDF documents into piecewise chunks of text for embedding purposes.

In [30]:
chunked_pdf_files_dict = {}
for pdf_file in pdf_files:
    pdf_location = DATAPATH + pdf_file + ".pdf"
    chunked_pdf = chunk_documents(
        documents= TextLoader(file_path=pdf_location, loader=PyPDFLoader),
        TextSplitter=RecursiveCharacterTextSplitter,
        chunk_size=512, # 512 is maximum Sequence lentgth for 
        chunk_overlap=20,
        separator=None,
        )
    chunked_pdf_files_dict[pdf_file] = chunked_pdf

In [31]:
print(chunked_pdf_files_dict[Patricia_permobil_manual][79].page_content)

3. The assistant at the rear of the chair is in control of this procedure. He or she must tilt the chair back to its balance point on the rear wheels. NEVER attempt to lift a wheelchair by lifting on any removable (detachable) parts, including upholstery and removable push handles or push handle grips. 
4. The second assistant at the front must firmly grasp a non-detachable part of the front frame (but NOT swing away hangers) with both hands and lift the chair up and over one stair at a time.


We can also see how our text chunk for a table looks like.

In [115]:
print(chunked_pdf_files_dict[caterpillar_10k][104].page_content)

agreement.  At select business units, we have hired certain highly specialized employees under employment contracts that specify a
term of employment, pay and other benefits.
 
Full-T ime Employees at Year-End
 2022 2021
Inside U.S. 48,200 44,300
Outside U.S. 60,900 63,400
Total 109,100 107,700
By Region:   
North America 48,700 44,700
EAME 16,900 17,600
Latin America 19,100 19,500
Asia/Pacific 24,400 25,900
Total 109,100 107,700


#### Performing the embedding of the chunked PDF documents
> **NB: The below code block takes around 100 minutes to complete for the four PDF's (100-150 pages each), so this has been prepared before the lecture. We have kept the code here for those interested, but `DO NOT RUN THE BELOW TWO CELLS!`**

In [33]:
embed_pdfs = False

# For embeddings we use the nr 1 model on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard
embedding_model = ["BAAI/bge-large-en-v1.5"]

if embed_pdfs:

  embedding_dict = {}
  for model_name in embedding_model:
    for pdf_file in pdf_files:
      print(f"Creating FAISS embedding for document {pdf_file} using - {model_name}")
      start = timer()
      embedding = doc_embedding(embedding_model=model_name)
      faiss_index = make_index_FAISS(chunked_documents=chunked_pdf_files_dict[pdf_file], embedding=embedding)
      end = timer()
      time_taken=end-start
      print(f"Done! Embedded vector index created after {time_taken/60:.2f} minutes")
      print("*"*25)
      embedding_dict[pdf_file] = faiss_index

Here we save the embeddings so that we don't have to rerun the above every time

In [34]:
save_embeddings = False
if save_embeddings:
    for pdf_embedding in embedding_dict.keys():
        FAISS_folder_name = DATAPATH + f"FAISS_{pdf_embedding}.faiss"
        embedding_dict[pdf_embedding].save_local(FAISS_folder_name)

### Loading the saved embeddings

If embeddings are already saved we can simply load them from storage.

In [35]:
faiss_embeddings_dict = {}
if not save_embeddings:
    for pdf_file in pdf_files:
        FAISS_folder_name = DATAPATH + f"FAISS_{pdf_file}.faiss"
        embedding = doc_embedding(embedding_model=embedding_model[0])
        faiss_embeddings_dict[pdf_file] = FAISS.load_local(FAISS_folder_name, embedding)

Let's try out making a simple query against the saved embeddings by doing a similarity search.

In [36]:
# We choose one of the PDF files and a relevant search query
pdf = Patricia_permobil_manual
similarity_query = "What is the procedure for setting up my TiLite rigid wheelchair?"

test = faiss_embeddings_dict[pdf].similarity_search(similarity_query, k=10)
print(f"Found on page: {test[0].metadata['page']}")
print(test[0].page_content)

Found on page: 78
11-4
ZR Owner’s Manual OM0005_Rev A_ZRCHAPTER 11: CASTERS AND FORKSNote: TiLite designs its rigid wheelchairs to be flexible for improved maneuverability and increased ride comfort.  
However, this flexibility requires that your chair be set up properly.  The following procedure will enable you to set up  your TiLite rigid wheelchair so it will perform to its potential.
1. Place the wheelchair on a smooth, level surface with the casters trailing rearward.


### Permobil Manual PDF

In [118]:
# This is the same as already defined above. We put it here again to allow you to change e.g. model or temperature more easily
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model='gpt-3.5-turbo', #'gpt-4'
    temperature=0.7
)

# We define a function for more easily call the LLM using the same prompt template but with a different query and context
def LLM_with_pdf_context(context: str, query: str, temperature: float=0.0):
    
    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful assistant and an expert in investment management.
    You are logical, methodical and a problem solving genius. \
    Always find the best and most simple and relevant solution to a problem. \
    Always break down the problem, objects, numbers and logic before starting to solve the problem. \
    Then solve the problem in a step-by-step manner.
    """

    # This prompt inouts the merged context
    user_prompt = f"""
    I need help from an investement management expert. \
    One of our portfolio companies have handed us these brief instruction snippets as
    context: {context}

    Use only the above context and nothing else to answer the following 
    question: {query}
    If the answer is provided in the form of a bulleted list within the context, \
        then return those instructions verbatim and do not try to rephrase them.
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]
    
    response = chat_model(messages, temperature=temperature)
    print(response.content)

> **We should do some testing on different questions here**

In [81]:
# We choose one of the PDF files and a relevant search query
pdf = Patricia_permobil_manual
query = "What tools are needed for adjusting the Front Seat Height on Slipstream Single-Sided Forks. Also provide the steps for performing the adjustment."

# Retrieve the most similar chunks related to above search query
faiss_index = faiss_embeddings_dict[pdf]
top_hits = similarity_search_FAISS(search_query=query, nr_hits=5, index_store=faiss_index)
context = "".join([document.page_content for document in top_hits])

First, let's use all of the retrieved top hits as context when querying the model.

In [82]:
# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

To adjust the Front Seat Height on Slipstream Single-Sided Forks, the following tools are needed:

- 5/8" Open End Wrench
- Screwdriver

The steps for performing the adjustment are as follows:

1. Remove the caster. Refer to "Slipstream Single-Sided Forks - Replacing Casters" on page 11-2 for instructions on how to remove the caster.
2. Depending on the fork that came with your chair, you may be able to adjust the Front Seat Height without changing the casters. Follow the procedures under "Standard Forks - Replacing Casters" on page 11-1 to mount the casters in the alternative axle holes in the fork.
3. Note that the full range of adjustability may not be available with 5" or 6" casters. Additional adjustability may be achieved with different forks or casters or with fork stem extensions.


We see that the instructions are incomplete. This is due to the fact that the most relevant chunk happened to only contain the first part of the full instructions. Since the chunks are small this is something that can happen quite often. In order to try to get a more complete response we can feed not the retrieved and merged chunks as context, but instead feed the whole PDF page of the most relevant hit, assuming it contains more fuller context. We can also do some combination of this and using other chunks if we find that top hit doesn't always work.

In [83]:
# Here we get the page of the top hit and use that whole PDF page as context
context_page = f"""{pdf_text[f'page_{top_hits[0].metadata["page"]}']}"""

# Use the pre-defined function but with the whole context page as context
LLM_with_pdf_context(context_page, query)

To adjust the Front Seat Height on Slipstream Single-Sided Forks, the following tools are needed:

- 5/8" Open End Wrench
- Screwdriver

The steps for performing the adjustment are as follows:

1. Remove the caster. Refer to "Slipstream Single-Sided Forks - Replacing Casters" on page 11-2 for instructions.
2. Using the shaft of the screwdriver, remove the E-Ring by pressing downward across the open portion of the E-Ring. Wear protective eyewear. See Figure 11-3.
3. Using the 5/8" Open End wrench, remove the axle from the Slipstream Single-Sided Fork.
4. Place the axle in the alternate axle hole and securely tighten.
5. Using the shaft of the screwdriver, replace the E-Ring by pressing downward across the closed portion of the E-Ring, snapping the E-ring into place. See Figure 11-3.
6. Replace the caster. Refer to "Slipstream Single-Sided Forks - Replacing Casters" on page 11-2 for instructions.
7. Follow Steps 1 through 6 on the opposite fork.

Please note the following warnings:

- Al

In this case this worked much better, since the retrieved page actually contained the full instructions.

### Electrolux Annual Report PDF

In [120]:
# Set our pdf variable to the Electrolux annual report
pdf = electrolux_ann_rep

In [84]:
#query = "Explain Electrolux's business in Latin America"
#query = "Which risk and challenges are mentioned in the report?"
query = "Which major events happened for Electrolux during 2022?"

# Use our function to retrieve the most similar chunks related to the search query for the given pdf
context = get_contexts(pdf=pdf, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

Based on the provided context, the major events that happened for Electrolux during 2022 are as follows:

- Electrolux announced a loss for the fourth quarter of 2022, with an estimated operating income of approximately SEK -2.0bn (0.9), including non-recurring items of SEK -1.4bn (-0.7).
- Electrolux decided to discontinue production at the Nyíregyháza factory in Hungary from the beginning of 2024.
- Electrolux divested its business in Russia and sold its Russian subsidiary on September 9, 2022, recording a capital loss of SEK 350m.
- Electrolux completed a share buyback program, repurchasing a total of 17,369,172 own series B shares for a total amount of SEK 3,032m.

Please note that the provided context may contain additional information that could be relevant to Electrolux's major events in 2022.


**Try it out yourself:**

Try it out by searching with your own query for information from the Electrolux Annual Report.

In [122]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."

context = get_contexts(pdf=pdf, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

LLM_with_pdf_context(context, query)

Based on the given context, it seems that there is a note repeated multiple times. The note consists of a sequence of numbers from 1 to 31, repeated five times.

To answer the question, we need to extract the bulleted list from the given context. Since the context is not provided in a clear format, we need to identify any patterns or structures that might indicate the presence of a bulleted list.

Upon analyzing the context, we can observe that the note is repeated five times, each time with the numbers 1 to 31. This repetition suggests that the bulleted list might be the numbers 1 to 31.

Therefore, the answer to the question is a bulleted list of the numbers 1 to 31:

- 1
- 2
- 3
- 4
- 5
- 6
- 7
- 8
- 9
- 10
- 11
- 12
- 13
- 14
- 15
- 16
- 17
- 18
- 19
- 20
- 21
- 22
- 23
- 24
- 25
- 26
- 27
- 28
- 29
- 30
- 31

Please note that this answer assumes that the bulleted list is indeed the numbers 1 to 31, as there is no other information provided in the given context.


### Caterpillar 10-K PDF

In [125]:
# Set our pdf variable to the Caterpillar 10-K report
pdf = caterpillar_10k

In [89]:
query = "Which identified risks and challenges are mentioned in the report?"

context = get_contexts(pdf=pdf, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

LLM_with_pdf_context(context, query)

Based on the provided context, the identified risks and challenges mentioned in the report are:

- Delays and increased costs resulting from supply chain challenges and availability issues with suppliers.
- Freight delays that could impact production in the company's facilities.
- Rising costs and the need for appropriate price actions in response.
- Fluctuations in freight costs, fuel costs, limitations on shipping and receiving capacity, and other disruptions in the transportation and shipping infrastructure.
- Exposure to multiple and potentially conflicting laws, regulations, and policies in different regions and countries.

Please note that the identified risks and challenges are mentioned in a paragraph format rather than a bulleted list.


**Tables in PDF-reports:**

The Caterpillar 10-K report contains multiple tables which may cause a problem when using the text chunks. Here we can try and ask questions about information found in tables in the report. However, depending on the chunks, the model may confuse numbers found in the tables.

In [91]:
query = "How many full-time employees did they have at the end of the year 2022 in Latin America?"

context = get_contexts(pdf=pdf, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

LLM_with_pdf_context(context, query)

To find the number of full-time employees in Latin America at the end of the year 2022, we need to locate the relevant information in the given context. 

From the provided context, we can see the following information about full-time employees at year-end:

Latin America:
2022: 19,100
2021: 19,500

Based on this information, the number of full-time employees in Latin America at the end of the year 2022 is 19,100.

Therefore, the answer to the question is: 
- Full-Time Employees at Year-End
  2022: 19,100


**Try it out yourself:**

Try it out by searching with your own query for information from the Caterpillar 10-K report.

In [ ]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."

context = get_contexts(pdf=pdf, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

LLM_with_pdf_context(context, query)

### Comparison: Electrolux Annual Report PDF and Whirlpool 10-K Report PDF

In [99]:
# We define a function for more easily call the LLM using the same prompt template but with a different query and contexts
def LLM_with_pdf_context_comparison(company1: str, company2: str, context1: str, context2: str, query: str, temperature: float=0.0):
    
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful assistant and an expert in investment management.
    You are logical, methodical and a problem solving genius. \
    Always find the best and most simple and relevant solution to a problem. \
    Always break down the problem, objects, numbers and logic before starting to solve the problem. \
    Then solve the problem in a step-by-step manner.
    """

    # This prompt inouts the merged context
    user_prompt = f"""
    I need help from an investement management expert. \
    Two of our portfolio companies have handed us these brief instruction snippets as
    contexts. Contexts for {company1}: {context1}, and for {company2}: {context2}

    Use only the above context and nothing else to answer the following 
    question: {query}
    If the answer is provided in the form of a bulleted list within the context, \
        then return those instructions verbatim and do not try to rephrase them.
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model(messages, temperature)
    print(response.content)

You can also use queries to compare the information in different pdf:s by creating separate contexts. Here, we compare the information in the Electrolux Annual report and Whirlpool 10-K report.

In [102]:
# Let's make a query which we would like to pose to both the Caterpillar and the Electrolux PDF files
query = "Which key business strategies were mentioned in the report?"

# Using the above query, we can fetch relevant contexts from both PDF files
context_whirlpool = get_contexts(pdf=whirlpool_10k, query=query, faiss_embeddings_dict=faiss_embeddings_dict)
context_electrolux = get_contexts(pdf=electrolux_ann_rep, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

In [108]:
# Given the above retrieved contexts we can now ask a comparative question between the companies
chat_query = "This is the key business strategies for Whirlpool and Electrolux. Compare them. Which strategies are similar and which are different?"

In [109]:
# Use our pre-defined function together with the names of the companies and their context
LLM_with_pdf_context_comparison("Whirlpool", "Electrolux", context_whirlpool, context_electrolux, chat_query)

The key business strategies for Whirlpool and Electrolux can be compared as follows:

Similar strategies:
- Both companies focus on investing in businesses that support high growth and high margins.
- They both emphasize the importance of digitalization and automation in providing cost-competitive products with high quality.
- Both companies target the middle class as a driving force for market growth in emerging markets.
- They both consider industry trends and market intelligence to develop an understanding of the prevailing business context.
- Both companies have financial targets for profitable growth and prioritize achieving these targets.
- They both aim to create and sustain value through their business strategies.

Different strategies:
- Whirlpool highlights three strong pillars: small appliances, major appliances in the Americas and India, and commercial appliances. Electrolux does not mention specific pillars but focuses on driving sustainable consumer experience innovation 

**Try it out yourself:**

Try it out by telling your model what information from the reports you want to compare, e.g., their sustainability strategies.

In [110]:
# Write your own query on what you like to pose to both the Caterpillar and the Electrolux PDF files
query = "..."

# Using the above query, we can fetch relevant contexts from both PDF files
context_whirlpool = get_contexts(pdf=whirlpool_10k, query=query, faiss_embeddings_dict=faiss_embeddings_dict)
context_electrolux = get_contexts(pdf=electrolux_ann_rep, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

# Change this query to ask the model to compare ... for the companies.
chat_query = "This is the ... for Whirlpool and Electrolux. Compare them."

# Use our pre-defined function together with the names of the companies and their context
LLM_with_pdf_context_comparison("Whirlpool", "Electrolux", context_whirlpool, context_electrolux, chat_query)

Based on the provided context, the following are the instructions for Whirlpool and Electrolux:

Whirlpool:
- Whirlpool uses Design for Sustainability principles in their global platforms and connects product sustainability directly with their business goals.
- They are committed to innovating for a new generation of consumers and have a world-class innovation pipeline.
- Whirlpool has set rigorous science-based targets for greenhouse gas reductions and other sustainability goals.
- They believe that an environmentally sustainable Whirlpool is a more competitive company in the long term.
- Whirlpool is committed to protecting the environment, supporting employee growth, and making their homes, communities, and operations better today and in the future.

Electrolux:
- Electrolux believes that sustainability leadership is crucial for long-term profitable growth.
- They have a consistent approach to sustainability in the countries where they operate.
- Sustainability is a key business dri

### Check accuracy of tests (OPTIONAL - MOSTLY FOR DEVELOPMENT TESTING AND WILL BE MODIFIED OR REMOVED)

**Below code block is implemented to do some tests for checking that similarity works as expected**

In [54]:
# loop through pdf, get a text snippet per page and make queries to check that the right page is retrieved
# Only included for testing that exact queries are working as expected
# This uses the chunking performed in previous section
queries = [chunked_pdf[idx].page_content[50:150] for idx in range(len(chunked_pdf))]
metadatas = [chunked_pdf[idx].metadata["page"] for idx in range(len(chunked_pdf))]


# if testing only one query for similarity
page_id = 78
exact_query = "The following procedure will enable you to set up your TiLite rigid wheelchair so it will perform to its potential."
similarity_query = "What is the procedure for setting up my TiLite rigid wheelchair?"

queries = [similarity_query]


# For embeddings we use the nr 1 model on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard
# Must be same as used for the saved embeddings
embedding_model = ["BAAI/bge-large-en-v1.5"]

scores = {}
for model in embedding_model:
  print(f"Creating FAISS embedding using - {model}")
  start = timer()
  embedding = doc_embedding(embedding_model=model)
  faiss_index = faiss_embeddings_dict[Patricia_permobil_manual]
  end = timer()
  time_taken=end-start
  print(f"Done!\n Embedded vector index created after {time_taken:.5f} [s]")
  accuracy = 0
  top5_accuracy = 0
  cnt=0
  print(f"Starting search using - {model}")
  start = timer()
  for search_query in queries:
    top_hits = similarity_search_FAISS(search_query=search_query, nr_hits=5, index_store=faiss_index)

    if top_hits[0].metadata["page"] == page_id: #metadatas[cnt]:
      accuracy+=1
    is_in_top_5, vec_hit_nr = search_result_is_in_top_5(docs=top_hits, id=page_id) #id=metadatas[cnt])
    if is_in_top_5:
      top5_accuracy+=1
    cnt+=1
  end = timer()
  time_taken=end-start
  print(f"Done!\n Searched through {len(queries)} queries in time: {time_taken:.5f} [s]")
  if len(queries) > 1:
    accuracy = accuracy/len(queries)
    top5_accuracy = top5_accuracy/len(queries)
  scores["model"] = model
  scores["accuracy"] = accuracy
  scores["top5_accuracy"] = top5_accuracy
  print(f"accuracy: {accuracy*100:.2f}%")
  print(f"top5_accuracy: {top5_accuracy*100:.2f}%, hit at vector nr {vec_hit_nr}")
  print("-"*50)

Creating FAISS embedding using - BAAI/bge-large-en-v1.5
Done!
 Embedded vector index created after 2.85235 [s]
Starting search using - BAAI/bge-large-en-v1.5
Done!
 Searched through 1 queries in time: 0.30055 [s]
accuracy: 100.00%
top5_accuracy: 100.00%, hit at vector nr 0
--------------------------------------------------


In [55]:
scores

{'model': 'BAAI/bge-large-en-v1.5', 'accuracy': 1, 'top5_accuracy': 1}

The below code snippet can be used during testing to make sure that we retrieve the correct pages for some example questions for which we know the correct answer.

In [56]:
# RAG test


#page_id = 73  # page index starts at 0
#page_id = 18  # page index starts at 0
page_id = 77  # page index starts at 0
#search_query = "What tools are needed to remove the wheel lock handle?"
#search_query = "What should you do to reduce the risk of the wheelchair tipping over?"
search_query = "What tools are needed for adjusting the Front Seat Height on Slipstream Single-Sided Forks. Also provide the steps for performing the adjustment."
queries = [search_query]

contexts = []
scores = {}
for model in embedding_model:
  print(f"Creating FAISS embedding using - {model}")
  start = timer()
  embedding = doc_embedding(embedding_model=model)
  #faiss_index = embedding_dict[model_name]
  faiss_index = faiss_embeddings_dict[Patricia_permobil_manual]
  end = timer()
  time_taken=end-start
  print(f"Done!\n Embedded vector index created after {time_taken:.5f} [s]")
  accuracy = 0
  top5_accuracy = 0
  cnt=0
  print(f"Starting search using - {model}")
  start = timer()
  for search_query in queries:
    top_hits = similarity_search_FAISS(search_query=search_query, nr_hits=5, index_store=faiss_index)
    context = "".join([document.page_content for document in top_hits])
    contexts.append(context)
    if top_hits[0].metadata["page"] == page_id: #metadatas[cnt]:
      accuracy+=1
    is_in_top_5, vec_hit_nr = search_result_is_in_top_5(docs=top_hits, id=page_id) #id=metadatas[cnt])
    if is_in_top_5:
      top5_accuracy+=1
    cnt+=1
  end = timer()
  time_taken=end-start
  print(f"Done!\n Searched through {len(queries)} queries in time: {time_taken:.5f} [s]")
  if len(queries) > 1:
    accuracy = accuracy/len(queries)
    top5_accuracy = top5_accuracy/len(queries)
  scores["model"] = model
  scores["accuracy"] = accuracy
  scores["top5_accuracy"] = top5_accuracy
  print(f"accuracy: {accuracy*100:.2f}%")
  print(f"top5_accuracy: {top5_accuracy*100:.2f}%, hit at vector nr {vec_hit_nr}")
  print("-"*50)

Creating FAISS embedding using - BAAI/bge-large-en-v1.5
Done!
 Embedded vector index created after 2.60682 [s]
Starting search using - BAAI/bge-large-en-v1.5
Done!
 Searched through 1 queries in time: 0.12784 [s]
accuracy: 100.00%
top5_accuracy: 100.00%, hit at vector nr 0
--------------------------------------------------


In [57]:
print(f"We have {len(contexts)} contexts, merged from {len(top_hits)} retrieved snippets, which can be fed to our LLM")
contexts[0][:100]

We have 1 contexts, merged from 5 retrieved snippets, which can be fed to our LLM


'11-3\nZR Owner’s Manual OM0005_Rev A_ZRCHAPTER 11: CASTERS AND FORKSAdjusting the Front Seat Height o'

#### End of the notebook

> **NB: After completed course we can flush and unmount drive**

In [58]:
# First flush and unmount drive after we are done,
# but to re-mount with new login we may need to remove dir manually first
if use_drive:
  drive.flush_and_unmount()
  !rm -rf /content/drive
#drive.mount('/content/drive', force_remount=True)